# PySpark Example of Pyspark ML

Based on **Krish Naik** video: https://www.youtube.com/watch?v=_C8kWso4ne4

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('ML').getOrCreate()

23/06/25 17:30:36 WARN Utils: Your hostname, Wiktors-MacBook-Air.local resolves to a loopback address: 127.0.0.1, but we couldn't find any external IP address!
23/06/25 17:30:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/25 17:30:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/25 17:30:38 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
# Read the dataset
training = spark.read.csv('data/test1.csv', header=True, inferSchema=True)

In [3]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'age', 'Experience', 'Salary']

## VectorAssembler

### Concept:



``` 
[Age, Experience]----> new feature----> independent feature
```

In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler = VectorAssembler(inputCols=["age", "Experience"], outputCol="Independent Features")

In [7]:
output = featureassembler.transform(training)

In [8]:
output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [9]:
output.columns

['Name', 'age', 'Experience', 'Salary', 'Independent Features']

In [10]:
finalized_data = output.select("Independent Features", "Salary")

In [11]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [19]:
from pyspark.ml.regression import LinearRegression
# train test split
train_data, test_data = finalized_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol = 'Independent Features', labelCol='Salary')
regressor = regressor.fit(train_data)

23/06/25 17:45:20 WARN Instrumentation: [6d2b5af3] regParam is zero, which might cause numerical instability and overfitting.
23/06/25 17:45:21 WARN Instrumentation: [6d2b5af3] Cholesky solver failed due to singular covariance matrix. Retrying with Quasi-Newton solver.


In [20]:
# Coefficients
regressor.coefficients

DenseVector([833.3333, 1250.0])

In [21]:
# Intercepts
regressor.intercept

-3749.9999999996494

In [22]:
# Prediction
pred_results = regressor.evaluate(test_data)

In [24]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000| 17916.66666666666|
|          [29.0,4.0]| 20000|25416.666666666562|
|          [30.0,8.0]| 25000| 31249.99999999987|
|         [31.0,10.0]| 30000|34583.333333333176|
+--------------------+------+------------------+



In [25]:
pred_results.meanAbsoluteError, pred_results.meanSquaredError

(4083.3333333332366, 22354166.666665614)